In [4]:
import openai, os, requests
import json
from dotenv import load_dotenv
load_dotenv()

api_type = os.getenv("OPENAI_API_TYPE")
# Azure OpenAI on your own data is only supported by the 2023-08-01-preview API version
api_version = os.getenv("OPENAI_API_VERSION")

# Azure OpenAI setup
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT") # Add your endpoint here
api_key = os.getenv("AZURE_OPENAI_API_KEY") # Add your OpenAI API key here
deployment_id = os.getenv("AZURE_OPENAI_MODEL_NAME") # Add your deployment ID here

# Azure AI Search setup
search_endpoint = os.getenv("SEARCH_ENDPOINT"); # Add your Azure AI Search endpoint here
search_key = os.getenv("SEARCH_KEY"); # Add your Azure AI Search admin key here
search_index_name = os.getenv("SEARCH_NAME"); # Add your Azure AI Search index name here

system_message = os.getenv("AZURE_OPENAI_SYSTEM_MESSAGE")



In [5]:
os.environ["OPENAI_API_TYPE"]

'azure'

In [27]:
client = openai.AzureOpenAI(
    base_url=f"{endpoint}/openai/deployments/{deployment_id}/extensions",
    api_key=api_key,
    api_version=api_version,
)

completion = client.chat.completions.create(
    model=deployment_id,
    messages=[
        {
            "role": "user",
            "content": "Who are you?",
        },
        {
        "role": "assistant",
        "content": "I'm Jason, an AI assistant employed by Manulife. I'm here to help you find information from the e-Agent Operating Manual (eAOM) source documents.",
        },
        {
            "role": "user",
            "content": "What can you tell me about TIA?",
        }
    ],
    stream=False,
    max_tokens=800,
    extra_body={
        "dataSources": [
            {
                "type": "AzureCognitiveSearch",
                "parameters": {
                    "endpoint": search_endpoint,
                    "key": search_key,
                    "indexName": search_index_name,
                    "inScope": True,
                    "topNDocuments": 2,
                    "queryType": "semantic",
                    "semanticConfiguration": "default",
                    "roleInformation": system_message,
                    "strictness": 3
                }
            }
        ]
    }
)

print(completion.model_dump_json(indent=2))

{
  "id": "cc4c97a8-5532-42b4-a4ba-3ab263ca6612",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The Temporary Insurance Agreement (TIA) is designed to provide temporary life insurance coverage to the proposed insured. It covers the period during which the insurance application is being underwritten by Manulife, starting at the time of completion of the application and subject to the conditions stated in the TIA[doc1].\n\nThe death benefit under the TIA will be paid by Manulife if the proposed insured dies due to causes not related to any material misrepresentation in the application form. However, the death benefit is not payable if the cause of death is suicide[doc1].\n\nThe TIA terminates on the earliest of the following: the date the policy applied for becomes effective, the date Manulife mails a notice to the applicant informing the termination of TIA, or 90 days from the sign date of the applica

In [25]:
print(completion.model_dump_json(indent=2))

{
  "id": "6689d815-9973-4a17-a11e-9b044380fff2",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "I'm Jason, an AI assistant employed by Manulife. I'm here to help you find information from the e-Agent Operating Manual (eAOM) source documents.",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null,
        "end_turn": true,
        "context": {
          "messages": [
            {
              "role": "tool",
              "content": "{\"citations\": [], \"intent\": \"[]\"}",
              "end_turn": false
            }
          ]
        }
      }
    }
  ],
  "created": 1709002330,
  "model": "gpt-4",
  "object": "extensions.chat.completion",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 36,
    "prompt_tokens": 3548,
    "total_tokens": 3584
  }
}


In [41]:
# python -m streamlit run src/frontend/EN_English.py --server.address 0.0.0.0 --server.port 8080

In [89]:
full_response = ""
for response in completion:
    print(response.choices[0])
    content = response.choices[0].delta.content
    full_response += content if content is not None else ""
    break
#print(full_response)

Choice(delta=ChoiceDelta(content=None, function_call=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None, end_turn=False)


In [88]:
first_r=response.copy()

/tmp/ipykernel_2366/7845261.py:1: PydanticDeprecatedSince20: The copy method is deprecated; use `model_copy` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  first_r=response.copy()


In [90]:
second_r = response.copy()

/tmp/ipykernel_2366/2435346865.py:1: PydanticDeprecatedSince20: The copy method is deprecated; use `model_copy` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  second_r = response.copy()


In [149]:
second_r.model_dump()['model']

'gpt-4'

In [111]:
json.loads(first_r.model_dump()['choices'][0]['delta']['context']['messages'][0]['content'])['citations']

[{'content': '8 Special Notes on Temporary Insurance Agreement (TIA)  \n \n\nTIA \n\nBackground & \nCoverage \n\n- TIA is to provide temporary life insurance coverage to the proposed insured. \n- It covers the period during which the insurance application is being underwritten \n\nby Manulife. The period starts at the time of completion of the application and \nsubject to the conditions stated in the TIA. \n\n- The amount of insurance under this agreement and all other temporary insurance \nagreements with the Company, please refer to “Terms” under TIA in details. \n\n \n\nPay the death \nbenefit \n\n- Manulife will pay the death benefit: \n➢ If the proposed insured die due to causes not related to any material \n\nmisrepresentation in the application form. \n➢ Death benefit is not payable if the cause of death is “suicide”',
  'id': None,
  'title': 'For Internal Reference Only Page 40',
  'filepath': 'page_040.pdf',
  'url': 'https://eaompocblob.blob.core.windows.net/split-pdfs/page_

In [97]:
second_r.model_dump()['choices'][0]['delta'].keys()

dict_keys(['content', 'function_call', 'role', 'tool_calls'])

In [54]:
response.model_dump().keys()

dict_keys(['id', 'choices', 'created', 'model', 'object', 'system_fingerprint'])

In [43]:
context_retrieved = json.loads(response.choices[0].delta.context['messages'][0]['content'])

In [47]:
print(context_retrieved['citations'][0]['content'])

8 Special Notes on Temporary Insurance Agreement (TIA)  
 

TIA 

Background & 
Coverage 

- TIA is to provide temporary life insurance coverage to the proposed insured. 
- It covers the period during which the insurance application is being underwritten 

by Manulife. The period starts at the time of completion of the application and 
subject to the conditions stated in the TIA. 

- The amount of insurance under this agreement and all other temporary insurance 
agreements with the Company, please refer to “Terms” under TIA in details. 

 

Pay the death 
benefit 

- Manulife will pay the death benefit: 
➢ If the proposed insured die due to causes not related to any material 

misrepresentation in the application form. 
➢ Death benefit is not payable if the cause of death is “suicide”
